# Bayesian Black-Box Optimisation (BBO) Capstone Project

**Imperial College Business School - Machine Learning & AI Programme**

This notebook implements Bayesian optimisation for eight unknown black-box functions, demonstrating the iterative process of querying, learning, and refining our search strategy.

## Table of Contents
1. [Setup & Imports](#1.-Setup-&-Imports)
2. [Understanding the Problem](#2.-Understanding-the-Problem)
3. [Load Initial Data](#3.-Load-Initial-Data)
4. [Gaussian Process Surrogate Model](#4.-Gaussian-Process-Surrogate-Model)
5. [Acquisition Functions](#5.-Acquisition-Functions)
6. [Optimisation Loop](#6.-Optimisation-Loop)
7. [Results Analysis](#7.-Results-Analysis)
8. [Generate Next Queries](#8.-Generate-Next-Queries)

## 1. Setup & Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel
from scipy.stats import norm
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

print("Libraries loaded successfully!")

## 2. Understanding the Problem

We have 8 black-box functions of varying dimensionality. Our goal is to **maximise** each function using limited queries.

In [ ]:
# Function metadata
FUNCTIONS = {
    1: {"dims": 2, "initial_points": 10, "analogy": "Radiation detection"},
    2: {"dims": 2, "initial_points": 10, "analogy": "Drug efficacy"},
    3: {"dims": 3, "initial_points": 15, "analogy": "Manufacturing quality"},
    4: {"dims": 4, "initial_points": 30, "analogy": "Process optimisation"},
    5: {"dims": 4, "initial_points": 20, "analogy": "Resource allocation"},
    6: {"dims": 5, "initial_points": 20, "analogy": "Side effect minimisation"},
    7: {"dims": 6, "initial_points": 30, "analogy": "Robot control"},
    8: {"dims": 8, "initial_points": 40, "analogy": "Complex system tuning"},
}

# Display function info
func_df = pd.DataFrame([
    {"Function": f"F{k}", **v} for k, v in FUNCTIONS.items()
])
print("Function Overview:")
display(func_df)

## 3. Load Initial Data

Load the `.npy` files provided for each function.

In [ ]:
def load_function_data(func_num, data_dir="../data/raw"):
    """Load data for a specific function."""
    import os
    func_dir = os.path.join(data_dir, f"function_{func_num}")
    X = np.load(os.path.join(func_dir, "initial_inputs.npy"))
    Y = np.load(os.path.join(func_dir, "initial_outputs.npy"))
    return X, Y.ravel()

# Example: Load Function 1 data (if available)
# X1, Y1 = load_function_data(1)
# print(f"Function 1: {X1.shape[0]} points, {X1.shape[1]} dimensions")
# print(f"Output range: [{Y1.min():.4f}, {Y1.max():.4f}]")

### Accumulated Results (Rounds 1-5)

Since we've completed 5 rounds, let's record all our observations:

In [ ]:
# Results tracking from all rounds
results = {
    "F1": {
        "dims": 2,
        "queries": [
            ("0.000000-0.999999", 0.0),           # R1
            ("0.999999-0.000000", 0.0),           # R2
            ("0.500000-0.500000", 2.68e-9),       # R3
            ("0.400000-0.600000", 1.45e-23),      # R4
            ("0.600000-0.400000", "pending"),     # R5
        ],
        "best": 2.68e-9,
    },
    "F2": {
        "dims": 2,
        "queries": [
            ("0.999999-0.050505", -0.069),
            ("0.750000-0.950000", 0.334),
            ("0.700000-0.930000", 0.651),
            ("0.690000-0.940000", 0.639),
            ("0.710000-0.920000", "pending"),
        ],
        "best": 0.651,
    },
    "F3": {
        "dims": 3,
        "queries": [
            ("0.999999-0.999999-0.000000", -0.161),
            ("0.500000-0.600000-0.200000", -0.122),
            ("0.490000-0.610000-0.340000", -0.033),
            ("0.485000-0.615000-0.345000", -0.031),
            ("0.480000-0.620000-0.350000", "pending"),
        ],
        "best": -0.031,
    },
    "F4": {
        "dims": 4,
        "queries": [
            ("0.400827-0.428837-0.398466-0.449225", -0.006),
            ("0.400000-0.430000-0.400000-0.450000", -0.037),
            ("0.410000-0.440000-0.410000-0.460000", -0.338),
            ("0.400000-0.428000-0.398000-0.449000", -0.003),
            ("0.398000-0.430000-0.400000-0.448000", "pending"),
        ],
        "best": -0.003,
    },
    "F5": {
        "dims": 4,
        "queries": [
            ("0.225929-0.841160-0.884257-0.888829", 1139.17),
            ("0.220000-0.850000-0.890000-0.900000", 1264.40),
            ("0.210000-0.860000-0.900000-0.920000", 1484.13),
            ("0.200000-0.870000-0.910000-0.950000", 1808.33),
            ("0.190000-0.880000-0.930000-0.980000", "pending"),
        ],
        "best": 1808.33,
    },
    "F6": {
        "dims": 5,
        "queries": [
            ("0.059926-0.042164-0.264883-0.999568-0.020803", -1.36),
            ("0.730000-0.150000-0.730000-0.700000-0.050000", -0.737),
            ("0.720000-0.140000-0.720000-0.710000-0.040000", -0.678),
            ("0.710000-0.130000-0.710000-0.720000-0.030000", -0.680),
            ("0.700000-0.100000-0.700000-0.730000-0.010000", "pending"),
        ],
        "best": -0.678,
    },
    "F7": {
        "dims": 6,
        "queries": [
            ("0.050000-0.500000-0.250000-0.200000-0.400000-0.750000", 1.353),
            ("0.055000-0.490000-0.245000-0.220000-0.420000-0.780000", 1.271),
            ("0.045000-0.510000-0.255000-0.180000-0.380000-0.720000", 1.390),
            ("0.040000-0.520000-0.265000-0.160000-0.360000-0.700000", 1.376),
            ("0.045000-0.510000-0.255000-0.180000-0.380000-0.720000", "pending"),
        ],
        "best": 1.390,
    },
    "F8": {
        "dims": 8,
        "queries": [
            ("0.069392-0.052126-0.009451-0.005710-0.560896-0.321453-0.013713-0.233966", 9.780),
            ("0.060000-0.040000-0.010000-0.010000-0.550000-0.350000-0.015000-0.250000", 9.787),
            ("0.055000-0.035000-0.008000-0.008000-0.540000-0.360000-0.012000-0.260000", 9.782),
            ("0.050000-0.030000-0.006000-0.006000-0.535000-0.365000-0.010000-0.265000", 9.777),
            ("0.055000-0.035000-0.008000-0.008000-0.545000-0.355000-0.012000-0.258000", "pending"),
        ],
        "best": 9.787,
    },
}

# Display summary
summary_data = []
for func, data in results.items():
    summary_data.append({
        "Function": func,
        "Dimensions": data["dims"],
        "Queries": len(data["queries"]),
        "Best Output": data["best"],
    })

summary_df = pd.DataFrame(summary_data)
print("\nResults Summary (After Round 5):")
display(summary_df)

## 4. Gaussian Process Surrogate Model

The GP provides predictions with uncertainty estimates, crucial for balancing exploration and exploitation.

In [ ]:
def create_gp_surrogate(length_scale=0.5, noise_level=0.1, n_restarts=10):
    """
    Create a Gaussian Process surrogate model.
    
    Args:
        length_scale: RBF kernel length scale
        noise_level: Observation noise level
        n_restarts: Number of optimizer restarts
        
    Returns:
        Configured GaussianProcessRegressor
    """
    kernel = (
        ConstantKernel(1.0, constant_value_bounds=(1e-3, 1e3)) * 
        RBF(length_scale=length_scale, length_scale_bounds=(1e-3, 1e1)) + 
        WhiteKernel(noise_level=noise_level, noise_level_bounds=(1e-10, 1e1))
    )
    
    gp = GaussianProcessRegressor(
        kernel=kernel,
        n_restarts_optimizer=n_restarts,
        normalize_y=True,
        random_state=42
    )
    
    return gp

print("GP surrogate model function defined.")

## 5. Acquisition Functions

These functions guide where to query next.

In [ ]:
def upper_confidence_bound(X, gp, beta=2.0):
    """
    UCB acquisition function: μ(x) + β * σ(x)
    Higher β → more exploration
    """
    mean, std = gp.predict(X, return_std=True)
    return mean + beta * std


def expected_improvement(X, gp, y_best, xi=0.01):
    """
    Expected Improvement: E[max(f(x) - y_best, 0)]
    """
    mean, std = gp.predict(X, return_std=True)
    std = np.maximum(std, 1e-9)
    
    z = (mean - y_best - xi) / std
    ei = (mean - y_best - xi) * norm.cdf(z) + std * norm.pdf(z)
    ei[std < 1e-9] = 0.0
    
    return ei


def format_query(x):
    """Format query for portal submission."""
    x = np.clip(x, 0.0, 0.999999)
    return "-".join([f"{val:.6f}" for val in x])


print("Acquisition functions defined.")

## 6. Optimisation Loop

The main Bayesian optimisation loop.

In [ ]:
def suggest_next_point(X, Y, bounds, acquisition="ucb", beta=2.0, n_random=1000):
    """
    Suggest the next query point using Bayesian optimisation.
    
    Args:
        X: Observed inputs
        Y: Observed outputs
        bounds: [(min, max), ...] for each dimension
        acquisition: 'ucb' or 'ei'
        beta: UCB exploration parameter
        n_random: Number of random candidates
        
    Returns:
        x_next: Suggested query point
        acq_value: Acquisition value
    """
    # Fit GP
    gp = create_gp_surrogate()
    gp.fit(X, Y)
    
    y_best = np.max(Y)
    n_dims = X.shape[1]
    bounds = np.array(bounds)
    
    # Random search
    X_random = np.random.uniform(
        bounds[:, 0], bounds[:, 1], 
        size=(n_random, n_dims)
    )
    
    # Compute acquisition values
    if acquisition == "ucb":
        acq_values = upper_confidence_bound(X_random, gp, beta=beta)
    else:
        acq_values = expected_improvement(X_random, gp, y_best)
    
    # Select best
    best_idx = np.argmax(acq_values)
    x_next = X_random[best_idx]
    
    return x_next, acq_values[best_idx], gp


print("Optimisation loop defined.")

## 7. Results Analysis

Visualise the optimisation progress.

In [ ]:
# Plot convergence for all functions
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

# Best values per round (cumulative best)
convergence_data = {
    "F1": [0, 0, 0, 2.68e-9, 2.68e-9],
    "F2": [0.611, 0.611, 0.611, 0.651, 0.651],
    "F3": [-0.035, -0.035, -0.035, -0.033, -0.031],
    "F4": [-4.03, -0.006, -0.006, -0.006, -0.003],
    "F5": [1088.86, 1139.17, 1264.40, 1484.13, 1808.33],
    "F6": [-0.71, -0.71, -0.71, -0.678, -0.678],
    "F7": [1.365, 1.365, 1.365, 1.390, 1.390],
    "F8": [9.60, 9.78, 9.787, 9.787, 9.787],
}

for i, (func, values) in enumerate(convergence_data.items()):
    ax = axes[i]
    rounds = ["Init", "R1", "R2", "R3", "R4"]
    
    ax.plot(rounds, values, 'b-o', linewidth=2, markersize=8)
    ax.fill_between(range(len(values)), values, alpha=0.3)
    
    # Highlight best
    best_idx = np.argmax(values)
    ax.scatter([best_idx], [values[best_idx]], color='red', s=150, marker='*', zorder=5)
    
    ax.set_title(f"{func} ({FUNCTIONS[i+1]['dims']}D)", fontsize=12)
    ax.set_xlabel("Round")
    ax.set_ylabel("Best Output")
    ax.grid(True, alpha=0.3)

plt.suptitle("Optimisation Progress: Best Output per Round", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig("../results/figures/convergence.png", dpi=150, bbox_inches='tight')
plt.show()

print("Convergence plot saved to results/figures/convergence.png")

### Key Observations

| Function | Trend | Notes |
|----------|-------|-------|
| **F1** | ❌ Struggling | Extremely weak signal, need more exploration |
| **F2** | ✅ Good | Found optimum region around (0.70, 0.93) |
| **F3** | ✅ Improving | Steady progress toward local optimum |
| **F4** | ✅ Excellent | Recovered from R3 misstep, near-zero output |
| **F5** | 🚀 Outstanding | +66% improvement, clear unimodal structure |
| **F6** | ✅ Plateau | Approaching optimum, diminishing returns |
| **F7** | ✅ Stable | Found good region, fine-tuning |
| **F8** | ✅ Plateau | Near-optimal, limited improvement possible |

## 8. Generate Next Queries

Generate and format queries for the next round submission.

In [ ]:
# Round 5 submissions (already submitted)
round_5_queries = {
    "F1": "0.600000-0.400000",
    "F2": "0.710000-0.920000",
    "F3": "0.480000-0.620000-0.350000",
    "F4": "0.398000-0.430000-0.400000-0.448000",
    "F5": "0.190000-0.880000-0.930000-0.980000",
    "F6": "0.700000-0.100000-0.700000-0.730000-0.010000",
    "F7": "0.045000-0.510000-0.255000-0.180000-0.380000-0.720000",
    "F8": "0.055000-0.035000-0.008000-0.008000-0.545000-0.355000-0.012000-0.258000",
}

print("="*60)
print("ROUND 5 SUBMISSIONS")
print("="*60)
for func, query in round_5_queries.items():
    print(f"\n{func}: {query}")

## Conclusion

After 5 rounds of Bayesian optimisation:

1. **F5** shows the most dramatic improvement (+66%), demonstrating successful exploitation of a unimodal structure
2. **F1** remains challenging—the signal is extremely weak, suggesting either a very narrow peak or sparse optimum
3. **F4** demonstrated the value of reverting to successful regions after failed exploration
4. Higher-dimensional functions (F6-F8) show slower but steady convergence

### Lessons Learned

- **Exploitation over exploration** proved more reliable with limited queries
- **Gradient information** from round-by-round changes guided effective adjustments
- **Reverting to successful regions** after exploration failures was crucial (F4, F7)

---

*Imperial College Business School - ML & AI Capstone Project*